<h3>Testy dla macierzy masy</h3>

In [24]:
def check_symmetric(M, rtol=1e-05, atol=1e-08):
    """
        Funkcja sprawdzająca czy macierz jest symetryczna,
        przy danej tolerancji.
        input: M - macierz do sprawdzenia, rtol, atol - tolerancje
    """

    if np.allclose(M, M.T, rtol=rtol, atol=atol):
        print("Macierz symetryczna")
    else:
        print("Macierz nie jest symetryczna")


def check_positive_definite(M, tol=1e-10):
    """
        Funkcja sprawdzająca czy macierz jest dodatnio określona,
        przy danej tolerancji.
        input: M - macierz do sprawdzenia, tol - tolerancja
        uwaga: jeśli wartości własne > 0 to macierz jest dodatnio określona
    """

    if np.all(np.linalg.eigvals(M) > -tol):
        print("Macierz dodatnio określona")
    else:
        print("Macierz nie jest dodatnio określona")


def check_cond(M):
    """
        Funkcja obliczająca uwarunkowanie macierzy.
        input: M - macierz do sprawdzenia, rtol i atol- tolerancje
    """

    print("test np.linalg.cond: ")
    try:
        cond = np.linalg.cond(M)
        if cond < 1/np.finfo(M.dtype).eps:
            print(
                "Macierz odwracalna, wyznacznik różny od zera,"
                "uwarunkowanie równe: ", cond)
        else:
            print("Macierz nie jest odwracalna, wyznacznik równy zero,"
                  "uwarunkowanie równe: ", cond)
    except Exception :
        print("nie udało się sprawdzić warunku uwarunkowania")


def test_mass_matrix(M, tol=1e-10, rtol=1e-05, atol=1e-08, h=1):
    """
        Szereg testów dla macierzy masy.
        Suma elementów macierzy masy musi być równa powierzchni obszaru,
        który reprezentuje,
        macierz masy jest symetryczna i dodatniookreślona.
    """

    try:
        M = M.toarray()  # z macierzy typu sparse do normalnej
    except Exception :
        pass
    print(M)
    check_symmetric(M, rtol, atol)
    check_positive_definite(M, tol)
    check_cond(M, rtol, atol)
    print("Suma liczb macierzy masy to ", sum(sum(M)),
          "musi zgadzać się z powierzchnią obszaru (bez dziur) "
          "jeśli jest to macierz masy globalna lub w przypadku "
          "macierzy masy lokalnej jest to powierzchnia elementu.")

ValueError: too many values to unpack (expected 3)

<h3> Testy dla macierzy sztywności </h3>

In [25]:
def test_stiffness_matrix(A, h=1, tol=1e-10, rtol=1e-05, atol=1e-08):
    """
        Szereg testów dla macierzy sztywności.
        A - macierz sztywnośći, h - globalny rozmiar siatki,
        pozostałe to opcjonalne tolerancje.

    """

    try:
        A = A.toarray()  # z macierzy typu sparse do normalnej
    except Exception:
        pass

    print(A)
    check_symmetric(A, rtol, atol)  # macierz sztywności jest symetryczna
    # i dodatnio określona
    check_positive_definite(A, tol)
    # suma elementów w dowolnej kolumnie, w dowolnym wierszu = 0
    print("sumujemy po kolumnach: ", np.sum(A, axis=0))
    check_cond(A, rtol, atol)  # wyznacznik macierzy sztywności jest równy 0
    # macierz sztywnosci nie ma zer, ani ujemnych wartosci na diagonali
    print("wartości na diagonali: ", np.diagonal(A))

<h3> Błąd w normie L2 </h3>

In [26]:
def L2_error(exact_sol, sol, M):
    """
        Funkcja do liczenia błędów w normie L^{2}.
        input:  exact_sol - np.array(), rozwiązanie dokładne;
                sol - np.array(), rozwiązanie przybliżone;
                M - macierz masy P1.
        output: błąd w normie L^{2}.
    """

    vector = exact_sol - sol
    error = M.dot(vector)
    error = Decimal(error.dot(vector)) ** (Decimal(1)/Decimal(2))

    return float(error)

<h3> Błąd w normie H1 </h3>

In [28]:
def H1_error(exact_sol1, sol1, exact_sol2, sol2, M, A):
    """
        Funkcja do liczenia błędów w normie H^{1}.
        input:  exact_sol1, exact_sol2 - np.array(), rozwiązania dokładne;
                sol1, sol2 - np.array(), rozwiązania przybliżone;
                M - macierz masy P2, A - macierz sztywności P2.
        output: błąd w normie H^{1}.
    """
    print(exact_sol1.shape)
    vector1 = exact_sol1 - sol1
    L2error1 = M.dot(vector1)
    L2error1 = Decimal(L2error1.dot(vector1))
    grad_error1 = A.dot(vector1)
    grad_error1 = Decimal(grad_error1.dot(vector1))

    vector2 = exact_sol2 - sol2
    L2error2 = M.dot(vector2)
    L2error2 = Decimal(L2error2.dot(vector2))
    grad_error2 = A.dot(vector2)
    grad_error2 = Decimal(grad_error2.dot(vector2))

    error = Decimal(L2error1 + grad_error1) ** (Decimal(1) / Decimal(2)) + \
        Decimal(L2error2 + grad_error2) ** (Decimal(1) / Decimal(2))

    return float(error)

<h3> Błędy dla kolejnych siatek </h3>

In [31]:
def calculate_errors(arr_nodes, arr_P2nodes,
                     arr_ksi_x, arr_ksi_y, arr_gamma,
                     arr_P1_M, arr_P2_M, arr_P2_A):
    """
        Funkcja do liczenia błędów, dla kolejnych siatek,
        względem najgęstszej siatki.
        input:  arr_nodes - list(), lista tablic węzłów P1;
                arr_P2nodes - list(), lista tablic węzłów P2;
                arr_ksi_x - list(), lista wektorów rozwiązań u1;
                arr_ksi_y - list(), lista wektorów rozwiązań u2;
                arr_gamma - list(), lista wektorów rozwiązań p;
                arr_P1_M - list(), lista macierzy masy P1;
                arr_P2_M - list(), lista macierzy masy P2;
                arr_P2_A - list(), lista macierzy sztywności P2.
        output: L2_errors - list(), lista błędów w normie L^{2};
                H1_errors - list(), lista błędów w normie H^{1}.

    """

    n = len(arr_nodes)  # liczba rozważanych siatek
    # liczba rozważanych wierzchołków interpolacji dla P1
    P1n_v = arr_nodes[0].shape[0]
    # liczba rozważanych wierzchołków interpolacji dla P2
    P2n_v = arr_P2nodes[0].shape[0]

    P1_ind = []  # lista list węzłów dla każdej siatki,
    # względem których liczymy błąd dla ciśnienia
    P2_ind = []  # lista list węzłów dla każdej siatki,
    # względem których liczymy błąd dla prędkości
    for i in range(n):
        P1_ind.append([])
        P2_ind.append([])

    L2_errors = []  # listy błędów dla kolejnych siatek
    H1_errors = []

    P1_M = arr_P1_M[0]  # macierze potrzebne do liczenia blędów
    P2_M = arr_P2_M[0]
    P2_A = arr_P2_A[0]

    # Znajdujemy numery węzłów, względem których liczymy błąd

    for i in range(arr_nodes[0].shape[0]):
        P1_ind[0].append(i)
    for i in range(arr_P2nodes[0].shape[0]):
        P2_ind[0].append(i)

    for k in range(1, n):

        for i in range(arr_nodes[0].shape[0]):
            for j in range(arr_nodes[k].shape[0]):
                if math.isclose(arr_nodes[0][i, 0], arr_nodes[k][j, 0]) and \
                        math.isclose(arr_nodes[0][i, 1], arr_nodes[k][j, 1]):
                    P1_ind[k].append(j)
                    break

        for i in range(arr_P2nodes[0].shape[0]):
            for j in range(arr_P2nodes[k].shape[0]):
                if math.isclose(arr_P2nodes[0][i, 0], arr_P2nodes[k][j, 0]) and \
                        math.isclose(arr_P2nodes[0][i, 1], arr_P2nodes[k][j, 1]):
                    P2_ind[k].append(j)
                    break

    # Błędy dla kolejnych siatek

    for k in range(0, n-1):
        L2_errors.append(L2_error(arr_gamma[n-1][P1_ind[n-1]],
                                  arr_gamma[k][P1_ind[k]], P1_M))

        H1_errors.append(H1_error(arr_ksi_x[n-1][P2_ind[n-1]],
                                  arr_ksi_x[k][P2_ind[k]],
                                  arr_ksi_y[n-1][P2_ind[n-1]],
                                  arr_ksi_y[k][P2_ind[k]], P2_M, P2_A))

    return L2_errors, H1_errors

59:80: E501 line too long (81 > 79 characters)
60:80: E501 line too long (81 > 79 characters)


<h3> Błąd w normie supremum </h3>
przepływ paraboliczny

In [33]:
def parabolic_flow_supremum_error(ksi_x, ksi_y, gamma, P1nodes, P2nodes):
    """
        Funkcja obliczająca błąd normy supremum dla poszczególnych
        wektorów rozwiązania przepływu funkcji parabolicznej.
        input: ksi_x - np.array(), wektor przybliżonego rozwiązania u_1;
        ksi_y - np.array(), wektor przybliżonego rozwiązania u_2;
        gamma - np.array(), wektor przybliżonego rozwiązania p;
        P1nodes - np.array(), lista węzłów dla ciśnienia;
        P2nodes - np.array(), ista węzłów dla prędkości.
    """

    real_ksi_x = np.zeros([P2nodes.shape[0]])
    real_ksi_y = np.zeros([P2nodes.shape[0]])
    real_gamma = np.zeros([P1nodes.shape[0]])

    # Obliczenie rozwiązań dokładnych

    for i in range(len(P2nodes)):
        real_ksi_x[i] = dirichlet_function(P2nodes[i, 1])

    for i in range(len(P1nodes)):
        real_gamma[i] = -2 * P2nodes[i, 0] + 2 * wall_right

    # Obliczenie błędów w normie supremum

    print('Błąd normy supremum wektora prędkości to ',
          max(abs(real_ksi_x-ksi_x)) + max(abs(real_ksi_y-ksi_y)),
          ' dla wartości ', real_ksi_x[np.argmax(max(abs(real_ksi_x-ksi_x))
                                                 + max(abs(real_ksi_y-ksi_y)))])
    print('Błąd normy supremum ciśnienia to ',
          max(abs(real_gamma-gamma)), ' dla wartości ',
          real_gamma[np.argmax(max(abs(real_gamma-gamma)))])

29:80: E501 line too long (80 > 79 characters)
